In [1]:
# Ensure python 3 compatibility
from __future__ import division, print_function, absolute_import

# Import necessary libraries:
# General utilities:
import sys
import os
import math

# Computation:
import numpy as np
import h5py

# Visualization:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import ipywidgets as widgets

from IPython.display import display

# Finally, pycroscopy itself
import pycroscopy as px

from os.path import join as pjoin

# set up notebook to show plots within the notebook
% matplotlib inline

import glob

#import moviepy.video.io.ImageSequenceClip
#try:
#    output = subprocess.check_output(['ffmpeg', '-version'])
#    version = output.split(b'\n')[0].split()[2]
#    print('Found: ffmpeg v{}'.format(version.decode('utf-8')))
#    ffmpeg_installed = True
#except:
 #   ffmpeg_installed = False

In [2]:
%%file custom.mplstyle

axes.linewidth: 1.5
xtick.major.size: 6
xtick.minor.size: 2
xtick.major.width: 1.5
xtick.minor.width: 1.5
ytick.major.size: 6
ytick.minor.size: 2
ytick.major.width: 1.5
ytick.minor.width: 1.5
axes.labelweight: bold
axes.labelpad: 1
axes.labelsize: 12
xtick.major.pad: 1
ytick.major.pad: 1
xtick.labelsize: 12
ytick.labelsize: 12
xtick.top: True
ytick.right: True
xtick.direction: in
ytick.direction: in
image.interpolation: nearest
    
# Loads the custom style
plt.style.use('./custom.mplstyle')

Overwriting custom.mplstyle


In [3]:
plt.style.use('./custom.mplstyle')

C:\Users\Josh_\Anaconda3\lib\site-packages\matplotlib\__init__.py:955: UserWarning: Illegal line #25
	"plt.style.use('./custom.mplstyle')"
	in file "./custom.mplstyle"
  warnings.warn('Illegal %s' % error_details)


In [4]:
 input_file_path ='D:/Scan_8_ckpfm/Doped_PZT_001_ckpfm_0015.h5' #px.io_utils.uiGetFile(caption='Select translated .h5 file or raw experiment data',
                                         #filter='Translated file (*.h5);; \\ Parameters for raw BE data (*.txt *.mat *xls *.xlsx)')

(data_dir, data_name) = os.path.split(input_file_path)

if input_file_path.endswith('.h5'):
    # No translation here
    h5_path = input_file_path
    tl = px.LabViewH5Patcher()
    hdf = tl.translate(h5_path)
else:
    # Set the data to be translated
    data_path = input_file_path

    (junk, base_name) = os.path.split(data_dir)

    # Check if the data is in the new or old format.  Initialize the correct translator for the format.
    if base_name == 'newdataformat':
        (junk, base_name) = os.path.split(junk)
        translator = px.BEPSndfTranslator(max_mem_mb=max_mem)
    else:
        translator = px.BEodfTranslator(max_mem_mb=max_mem)
    if base_name.endswith('_d'):
        base_name = base_name[:-2]
    # Translate the data
    h5_path = translator.translate(data_path, show_plots=True, save_plots=False)
    tl = px.LabViewH5Patcher()
    hdf = tl.translate(h5_path)
print('Working on:\n' + h5_path)

h5_main = px.hdf_utils.getDataSet(hdf.file, 'Raw_Data')[0]

File is already Pycroscopy ready.
Working on:
D:/Scan_8_ckpfm/Doped_PZT_001_ckpfm_0015.h5


In [29]:
h5_sho_group = px.hdf_utils.findH5group(h5_main, 'SHO_Fit')
sho_fitter = px.BESHOmodel(h5_main, parallel=True)
if len(h5_sho_group) == 0:
    print('No SHO fit found. Doing SHO Fitting now')
    h5_sho_guess = sho_fitter.do_guess(strategy='complex_gaussian', processors=max_cores)
    h5_sho_fit = sho_fitter.do_fit(processors=max_cores)
else:
    print('Taking previous SHO results already present in file')
    h5_sho_guess = h5_sho_group[-1]['Guess']
    try:
        h5_sho_fit = h5_sho_group[-1]['Fit']
    except KeyError:
        print('Previously computed guess found. Now computing fit')
        h5_sho_fit = sho_fitter.do_fit(processors=max_cores, h5_guess=h5_sho_guess)

Taking previous SHO results already present in file


In [6]:
# Specify output file path
output_file_path = './'

# If HV amplifier was used set high_voltage_amplf to 10, else to 1
high_voltage_amplf = 10


In [30]:
(Nd_mat, success) = px.io.hdf_utils.reshape_to_Ndims(h5_sho_fit)
print('Reshape Success = ' + str(success))
print('Nd_mat shape = ', Nd_mat.shape)

phase_offset = Nd_mat[0, 0, 1, 0, 0]['Phase [rad]']

# phase_offset = 0;

print('Phase offset [rad] = ', phase_offset)

Nd_mat[:,:,:,:,:]['Phase [rad]'] = Nd_mat[:,:,:,:,:]['Phase [rad]'] - phase_offset

Reshape Success = True
Nd_mat shape =  (70, 70, 2, 84, 11)
Phase offset [rad] =  0.199172


In [28]:
Nd_mat[:1,1,1,1,1]

array([( 0.00311499,  325349.59375,  305.64855957,  0.16913097,  0.90679371)],
      dtype=[('Amplitude [V]', '<f4'), ('Frequency [Hz]', '<f4'), ('Quality Factor', '<f4'), ('Phase [rad]', '<f4'), ('R2 Criterion', '<f4')])

In [31]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from scipy import io
import numpy as np
import pycroscopy as px
import matplotlib.pyplot as plt
from os.path import join as pjoin
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential, Input, Model
from keras.layers import (Dense, Conv1D, Convolution2D, GRU, LSTM, Recurrent, Bidirectional, TimeDistributed,
                          Dropout, Flatten, RepeatVector, Reshape, MaxPooling1D, UpSampling1D, BatchNormalization)
from keras.layers.core import Lambda
from keras.optimizers import Adam
import os
from os.path import join as pjoin
from keras.regularizers import l1
import sys


% matplotlib inline

In [ ]:
len(qf_on.reshape(-1))

In [10]:
for i in range(len(qf_on.reshape(-1))):
    if qf_on.reshape(-1)[i] > 900:
        qf_on.reshape(-1)[i] = np.mean(qf_on)
        
for i in range(len(qf_off.reshape(-1))):
    if qf_off.reshape(-1)[i] > 900:
        qf_off.reshape(-1)[i] = np.mean(qf_off)

In [8]:
def norm(indat):
    indat-=np.mean(indat)
    indat/=np.std(indat)
    return indat

In [ ]:
plt.hist(qf_on.reshape(-1),100)

In [ ]:
np.std((Nd_mat["Amplitude [V]"][:,:,:,1,2].reshape(-1)))

In [9]:
amp_off = np.atleast_3d(Nd_mat["Amplitude [V]"][:,:,:,1,2].reshape(-1,128))
phase_off = np.atleast_3d(Nd_mat["Phase [rad]"][:,:,:,1,2].reshape(-1,128))
freq_off = np.atleast_3d(Nd_mat["Frequency [Hz]"][:,:,:,1,2].reshape(-1,128))
qf_off = np.atleast_3d(Nd_mat["Quality Factor"][:,:,:,1,2].reshape(-1,128))

amp_on = np.atleast_3d(Nd_mat["Amplitude [V]"][:,:,:,0,2].reshape(-1,128))
phase_on = np.atleast_3d(Nd_mat["Phase [rad]"][:,:,:,0,2].reshape(-1,128))
freq_on = np.atleast_3d(Nd_mat["Frequency [Hz]"][:,:,:,0,2].reshape(-1,128))
qf_on= np.atleast_3d(Nd_mat["Quality Factor"][:,:,:,0,2].reshape(-1,128))

In [11]:
amp_off = np.atleast_3d(norm(amp_off))
phase_off = np.atleast_3d(norm(phase_off))
freq_off = np.atleast_3d(norm(freq_off))
qf_off = np.atleast_3d(norm(qf_off))

amp_on = np.atleast_3d(norm(amp_on))
phase_on = np.atleast_3d(norm(phase_on))
freq_on = np.atleast_3d(norm(freq_on))
qf_on= np.atleast_3d(norm(qf_on))

In [12]:
def make_folder(folder_name, root='./'):

    folder = pjoin(root, '{}'.format(folder_name))
    os.makedirs(folder, exist_ok=True)

    return (folder)

In [13]:
def encode_layer(size, numbernodes,x):
    for i in range(size-1):
        x = Bidirectional(LSTM(numbernodes, return_sequences=True))(x)
    out = Bidirectional(LSTM(numbernodes, return_sequences=False))(x)
    return out

In [14]:
def decode_layer(size, numbernodes,x):
    x = RepeatVector(128)(x)
    for i in range(size):
        x = Bidirectional(LSTM(numbernodes, return_sequences=True))(x)
    x = TimeDistributed(Dense(1, activation='linear'))(x)
    return x

In [15]:
input_on_amp = Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_off_amp =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_on_phase =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_off_phase =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_on_resonance =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_off_resonance = Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_on_loss =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_off_loss =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))


en_out_a_on = encode_layer(3,16,input_on_amp)
en_out_a_on = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_a_on)
en_out_a_off = encode_layer(3,16,input_off_amp)
en_out_a_off = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_a_off)
en_out_p_on = encode_layer(3,16,input_on_phase)
en_out_p_on = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_p_on)
en_out_p_off = encode_layer(3,16,input_off_phase)
en_out_p_off = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_p_off)
en_out_r_on = encode_layer(3,16,input_on_resonance)
en_out_r_on = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_r_on)
en_out_r_off = encode_layer(3,16,input_off_resonance)
en_out_r_off = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_r_off)
en_out_l_on = encode_layer(3,16,input_on_loss)
en_out_l_on = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_l_on)
en_out_l_off = encode_layer(3,16,input_off_loss)
en_out_l_off = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_l_off)



x = keras.layers.concatenate([en_out_a_on, en_out_a_off, en_out_p_on, en_out_p_off,
                              en_out_r_on, en_out_r_off, en_out_l_on, en_out_l_off])

x = Dense(12, activation='relu',activity_regularizer=l1(10e-4))(x)

de_out_a_on = decode_layer(3,16,x)
de_out_a_off =decode_layer(3,16,x)
de_out_p_on = decode_layer(3,16,x)
de_out_p_off =decode_layer(3,16,x)
de_out_r_on = decode_layer(3,16,x)
de_out_r_off =decode_layer(3,16,x)
de_out_l_on = decode_layer(3,16,x)
de_out_l_off =decode_layer(3,16,x)

model = Model(inputs=[input_on_amp, input_off_amp, input_on_phase, input_off_phase, input_on_resonance,
                      input_off_resonance, input_on_loss, input_off_loss], 
              outputs=[de_out_a_on, de_out_a_off, de_out_p_on, de_out_p_off, de_out_r_on, de_out_r_off,
                       de_out_l_on, de_out_l_off])

model.compile(optimizer=Adam(3e-5),loss='mse')#

In [ ]:
Path = 'try6'
make_folder(Path)
tbCallBack = keras.callbacks.TensorBoard(
    log_dir='./' + Path, histogram_freq=0, write_graph=True, write_images=True)
filepath = './' + Path + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, verbose=0, save_best_only=False,
                                             save_weights_only=False, mode='min', period=1)
from keras.utils import multi_gpu_model

# Replicates `model` on 2 GPUs.
# This assumes that your machine has 2 available GPUs.
#parallel_model = multi_gpu_model(model, gpus=2)
#parallel_model.compile(optimizer=Adam(3e-5),loss='mse')
model.fit([amp_off,phase_off,freq_off,qf_off, amp_on, phase_on, freq_on, qf_on], 
          [amp_off,phase_off,freq_off,qf_off, amp_on, phase_on, freq_on, qf_on],
          validation_data=([amp_off,phase_off,freq_off,qf_off, amp_on, phase_on, freq_on, qf_on], 
          [amp_off,phase_off,freq_off,qf_off, amp_on, phase_on, freq_on, qf_on]),
          epochs=20000,batch_size=1280, callbacks=[tbCallBack, checkpoint])


Train on 6400 samples, validate on 6400 samples
Train on 6400 samples, validate on 6400 samples
Epoch 1/20000
Epoch 1/20000
6400/6400 [==============================] - 173s 27ms/step - loss: 9.0316 - time_distributed_1_loss: 0.9985 - time_distributed_2_loss: 1.0004 - time_distributed_3_loss: 0.9909 - time_distributed_4_loss: 1.0083 - time_distributed_5_loss: 0.9978 - time_distributed_6_loss: 1.0011 - time_distributed_7_loss: 1.0115 - time_distributed_8_loss: 1.0043 - val_loss: 8.9808 - val_time_distributed_1_loss: 0.9984 - val_time_distributed_2_loss: 1.0002 - val_time_distributed_3_loss: 0.9911 - val_time_distributed_4_loss: 1.0076 - val_time_distributed_5_loss: 0.9977 - val_time_distributed_6_loss: 1.0008 - val_time_distributed_7_loss: 1.0102 - val_time_distributed_8_loss: 1.0039
Epoch 2/20000
Epoch 2/20000
3840/6400 [=================>............] - ETA: 48s - loss: 8.9252 - time_distributed_1_loss: 0.9988 - time_distributed_2_loss: 1.0002 - time_distributed_3_loss: 0.9744 - time_

In [ ]:
model.save('./' + Path + '/end')

In [ ]:
dc_vec[0:128].shape

In [ ]:
model.summary()

In [ ]:
e = Nd_mat["Amplitude [V]"][:,:,:,1,2].reshape(-1,128)

h5_sho_spec_inds = px.hdf_utils.getAuxData(h5_sho_fit, auxDataName='Spectroscopic_Indices')[0]
h5_sho_spec_vals = px.hdf_utils.getAuxData(h5_sho_fit, auxDataName='Spectroscopic_Values')[0]

voltage_reshape = h5_sho_spec_vals[1] + h5_sho_spec_vals[2]

voltage_reshape.shape

dc_vec=dc_vec[::124]
plt.plot(dc_vec[0:128],amp_off[100])

In [ ]:
h5_file = h5_main.file
h5_spec_vals = h5_file[px.io.hdf_utils.get_attr(h5_main, 'Spectroscopic_Values')]
 
dc_vec = np.squeeze(h5_spec_vals[h5_spec_vals.attrs['DC_Offset']])


In [ ]:


model = Model(inputs=input_off_amp, outputs=out)


In [ ]:
type(e)

In [ ]:
model.summary()

In [ ]:
from keras.utils import multi_gpu_model

# Replicates `model` on 8 GPUs.
# This assumes that your machine has 8 available GPUs.
parallel_model = multi_gpu_model(model, gpus=8)
parallel_model.compile(loss='categorical_crossentropy',
                       optimizer='rmsprop')

# This `fit` call will be distributed on 8 GPUs.
# Since the batch size is 256, each GPU will process 32 samples.
parallel_model.fit(x, y, epochs=20, batch_size=256)